In [2]:
%%capture
%run full_setup.py

In [3]:
from tfidf_corpus_dictionary import get_tfidf_tokendocs_corpus_dict
from gensim.models import LdaModel, LsiModel, CoherenceModel
from sklearn.decomposition import NMF, PCA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
import numpy as np
from scipy import sparse

With this function we get various objects needed for modelling:
1. TFIDF matrix as input data, with specified parameters
2. feature names as the words retained with TFIDF
3. tokenized documents, a list of lists, where the inner lists contain the tokens for each document
4. corpus, gensim object needed for modelling with that package
4. dictionary, gensim object containing informations on the words of the corpus and their positions

In [4]:
tfidf_matrix, feature_names, tokenized_docs, corpus, dictionary = get_tfidf_tokendocs_corpus_dict(df, max_df=0.5, min_df=5, max_features=5000)

Now we'll evaluate different topic models based on Coherence score.

Coherence is a metric used to evalute topics quality. The higher the coherence, the better the model did in creating the topics.

For every model we'll use a function to retrieve coherence for different numbers of topics (5, 10, 15, 20, 50). This information will be used to evaluate how the models performed as the number of topics changes

In [5]:
from coherence_topics import coherence_topics

In [6]:
evaluation = dict()
models = ['LDA', 'LSA', 'NMF', 'PCA', 'RP']

for mod in models:
    metrics = coherence_topics(model_name=mod, corpus=corpus, dictionary=dictionary,
                               texts=tokenized_docs, feature_names=feature_names, tfidf=tfidf_matrix)
    evaluation[mod] = metrics

In [7]:
evaluation['LDA']

[(5, 0.545979304624336),
 (10, 0.5035821857797423),
 (15, 0.5285162980465483),
 (20, 0.5134451552900542),
 (50, 0.4640330668691648)]

In [8]:
evaluation['LSA']

[(5, 0.4982374193872543),
 (10, 0.43125932777615217),
 (15, 0.43098689001128787),
 (20, 0.40159132018753707),
 (50, 0.34832680646454284)]

In [15]:
evaluation['NMF']

[(5, 0.7147033104403053),
 (10, 0.7358479263967541),
 (15, 0.7352788685093538),
 (20, 0.7255836088832905),
 (50, 0.6186516384203021)]

In [17]:
evaluation['PCA']

[(5, 0.6217501371396356),
 (10, 0.5371818041799977),
 (15, 0.4748500960404024),
 (20, 0.4331432310448039),
 (50, 0.34502431736201217)]

In [14]:
evaluation['RP']

[(5, 0.5388460051693942),
 (10, 0.5435245527262123),
 (15, 0.5373158420089561),
 (20, 0.5206040179267396),
 (50, 0.5219495119538514)]

Now we'll fit the LDA model with the number of topics that yields the highest coherence

In [9]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5,
                     alpha='symmetric', eta='auto', passes=5, random_state=1)

In [10]:
for topic in lda_model.print_topics(num_words=15):
    topic_index, words = topic
    word_list = [word.split("*")[1].strip().strip('"') for word in words.split(" + ")]
    print(f"Topic {topic_index}: {', '.join(word_list)}")

Topic 0: car, drive, bike, sale, buy, speed, sell, price, distribution, game, motorcycle, usa, scsi, power, work
Topic 1: window, file, email, system, key, program, chip, computer, help, phone, software, version, work, problem, card
Topic 2: god, christian, jesus, bible, christ, religion, morality, christianity, church, faith, objective, believe, sin, truth, belief
Topic 3: beast, anthony, walker, mask, amanda, duo, murphy, pt, intercon, hamburg, singapore, cs, alex, vesselin, stats
Topic 4: government, state, year, israel, israeli, armenian, law, writes, kill, world, gun, child, case, david, post


In [11]:
import pyLDAvis, pyLDAvis.gensim

In [12]:
# Visualize the LDA model using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word, mds='tsne')
pyLDAvis.display(vis)

Now we're going to do the same for LSA

In [13]:
lsi_model = LsiModel(corpus, id2word=dictionary, num_topics=5)

C:\Users\sofch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\gensim\models\lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
C:\Users\sofch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\linalg\_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
C:\Users\sofch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\linalg\_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v

In [14]:
for topic in lsi_model.print_topics():
    topic_index, words = topic
    word_list = [word.split("*")[1].strip().strip('"') for word in words.split(" + ")]
    print(f"Topic {topic_index}: {', '.join(word_list)}")

Topic 0: system, work, window, god, problem, year, drive, distribution, file, computer
Topic 1: god, window, file, card, drive, christian, jesus, driver, believe, program
Topic 2: god, game, team, window, player, file, play, year, win, christian
Topic 3: key, chip, god, clipper, encryption, government, window, game, escrow, jesus
Topic 4: drive, window, file, car, scsi, game, team, program, ide, key
